In [10]:
import cv2
import iris
from pathlib import Path
from itertools import combinations

def load_template(img_path):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    result = iris.IRISPipeline()(img, eye_side="left")['iris_template']
    print(f'load template from image: {type(result)}')
    return result


In [11]:

dataset_path = Path("dataset")
matcher = iris.HammingDistanceMatcher()
threshold = 0.37
NUM_FOLDERS = 3

templates = {}
skipped_counts = {}  # Track skipped templates per folder

# Load templates
for folder in sorted(dataset_path.iterdir())[:NUM_FOLDERS]:
    if not folder.is_dir():
        continue
    print(f"Loading images from {folder.name}")
    folder_templates = []
    skipped = 0
    for img_path in folder.glob("*.jpg"):
        try:
            template = load_template(img_path)
            if template is not None:
                folder_templates.append(template)
            else:
                skipped += 1
        except Exception as e:
            print(f"  Failed to process {img_path.name}: {e}")
            skipped += 1
    templates[folder.name] = folder_templates
    skipped_counts[folder.name] = skipped
    print(f"  Successfully loaded {len(folder_templates)} templates, skipped {skipped} images")


Loading images from 000
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'NoneType'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
  Successfully loaded 9 templates, skipped 1 images
Loading images from 001
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'NoneType'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.IrisTemplate'>
out: <class 'iris.io.dataclasses.Iri

In [7]:
templates['000'][0]

IrisTemplate(iris_codes=[array([[[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [False,  True],
        [ True,  True],
        [ True,  True]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True,  True],
        [ True,  True],
        [ True,  True]],

       [[ True, False],
        [False, False],
        [False,  True],
        ...,
        [ True,  True],
        [ True,  True],
        [ True, False]],

       ...,

       [[False,  True],
        [False,  True],
        [False,  True],
        ...,
        [False, False],
        [False, False],
        [False, False]],

       [[False,  True],
        [False,  True],
        [False,  True],
        ...,
        [False, False],
        [False, False],
        [False, False]],

       [[False,  True],
        [False,  True],
        [False,  True],
        ...,
        [False, False],
        [False, False],
        [False, False]]]), array([

In [4]:
num_false_diffs = 0   
num_correct_matches = 0

# Compare within folder
print("\nComparing within the same subject:")
for name, tmpl_list in templates.items():
    for t1, t2 in combinations(tmpl_list, 2):
        if t1 is None or t2 is None:
            continue
        print(f't1: {type(t1)}')
        print(f't2: {type(t2)}')
        d = matcher.run(t1, t2)
        print(f"  {name}: distance = {d:.4f} => {'Match' if d < threshold else 'No Match'}")
        if d < threshold:
            num_correct_matches += 1
        else:
            num_false_diffs += 1

print(f"Number of false differences: {num_false_diffs}")
print(f"Number of correct matches: {num_correct_matches}")


Comparing within the same subject:
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.2888 => Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.3809 => No Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.2236 => Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.4350 => No Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.3890 => No Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.3796 => No Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <class 'iris.io.dataclasses.IrisTemplate'>
  000: distance = 0.3590 => Match
t1: <class 'iris.io.dataclasses.IrisTemplate'>
t2: <clas

In [20]:
num_false_matches = 0   
num_correct_rejects = 0

# Compare across folders
print("\nComparing across different subjects:")
for (name1, tmpl_list1), (name2, tmpl_list2) in combinations(templates.items(), 2):
    for t1 in tmpl_list1:
        for t2 in tmpl_list2:
            if t1 is None or t2 is None:
                continue
            d = matcher.run(t1, t2)
            print(f"  {name1} vs {name2}: distance = {d:.4f} => {'False Match!' if d < threshold else 'Correct Reject'}")
            if d < threshold:
                num_false_matches += 1
            else:
                num_correct_rejects += 1

print(f"Number of false matches: {num_false_matches}")
print(f"Number of correct rejects: {num_correct_rejects}")


Comparing across different subjects:
  000 vs 001: distance = 0.4761 => Correct Reject
  000 vs 001: distance = 0.4796 => Correct Reject
  000 vs 001: distance = 0.4740 => Correct Reject
  000 vs 001: distance = 0.4650 => Correct Reject
  000 vs 001: distance = 0.4642 => Correct Reject
  000 vs 001: distance = 0.4733 => Correct Reject
  000 vs 001: distance = 0.4785 => Correct Reject
  000 vs 001: distance = 0.4530 => Correct Reject
  000 vs 001: distance = 0.4676 => Correct Reject
  000 vs 001: distance = 0.4621 => Correct Reject
  000 vs 001: distance = 0.4598 => Correct Reject
  000 vs 001: distance = 0.4439 => Correct Reject
  000 vs 001: distance = 0.4648 => Correct Reject
  000 vs 001: distance = 0.4536 => Correct Reject
  000 vs 001: distance = 0.4647 => Correct Reject
  000 vs 001: distance = 0.4635 => Correct Reject
  000 vs 001: distance = 0.4701 => Correct Reject
  000 vs 001: distance = 0.4689 => Correct Reject
  000 vs 001: distance = 0.4735 => Correct Reject
  000 vs 001